Students Exam Scores: Extended Dataset | Kaggle
https://www.kaggle.com/datasets/desalegngeb/students-exam-scores

Проаналізуйте розподіл за статтю серед учнів.
1. Скільки студентів і студенток у наборі даних?
2. Який відсоток студентів чоловічої статі в наборі даних?
3. Який відсоток студенток у наборі даних?

In [3]:
import pandas as pd

# Path to your CSV file
file_path = 'Expanded_data_with_more_features.csv'

# Load the dataset
data = pd.read_csv(file_path)

In [5]:
gender_counts = data['Gender'].value_counts()

total_students = len(data)
percent_male = (gender_counts.get('male', 0) / total_students) * 100
percent_female = (gender_counts.get('female', 0) / total_students) * 100

gender_counts, percent_male, percent_female

(Gender
 female    15424
 male      15217
 Name: count, dtype: int64,
 49.6622172905584,
 50.337782709441605)

Дослідіть зв’язок між етнічною групою та результатами тестів.

4. Яка етнічна група має найвищий середній бал з математики?
5. Яка етнічна група має найвищий середній бал читання?
6. Яка етнічна група має найвищий середній бал з письма?

In [11]:
filtered_data = data.dropna(subset=['EthnicGroup'])

average_scores_by_ethnic_group = filtered_data.groupby('EthnicGroup')[['MathScore', 'ReadingScore', 'WritingScore']].mean()

average_scores_by_ethnic_group

,MathScore,ReadingScore,WritingScore
EthnicGroup,,,
group A,62.991888,66.787742,65.251915
group B,63.490216,67.320460,65.895125
group C,64.695723,68.438233,66.999240
group D,67.666400,70.382247,70.890844
group E,75.298936,74.251423,72.677060


In [13]:
highest_math_group = average_scores_by_ethnic_group['MathScore'].idxmax()
highest_reading_group = average_scores_by_ethnic_group['ReadingScore'].idxmax()
highest_writing_group = average_scores_by_ethnic_group['WritingScore'].idxmax()

highest_math_group, highest_reading_group, highest_writing_group

('group E', 'group E', 'group E')

Дослідіть вплив курсу підготовки до тесту на результати тесту.

7. Який середній бал з математики для студентів, які закінчили курс підготовки до тесту?
8. Який середній бал з читання для студентів, які пройшли курс підготовки до тесту?
9. Який середній бал для студентів, які закінчили курс підготовки до тесту?

In [14]:
test_prep_completed = data[data['TestPrep'] == 'completed']

average_scores_test_prep = test_prep_completed[['MathScore', 'ReadingScore', 'WritingScore']].mean()

average_scores_test_prep

MathScore       69.546660
ReadingScore    73.732998
WritingScore    74.703265
dtype: float64

Проаналізуйте розподіл батьків(ів) за сімейним станом серед учнів.

10. Скільки студентів мають одружених батьків?
11. Скільки студентів мають батьків-одинаків?

In [15]:
parental_marital_status_counts = data['ParentMaritalStatus'].value_counts()

parental_marital_status_counts[['married', 'single']]

ParentMaritalStatus
married    16844
single      7097
Name: count, dtype: int64

Дослідіть зв’язок між заняттями спортом і результатами тестів.

12. Чи студенти, які регулярно займаються спортом, мають вищі середні результати з математики, ніж ті, хто ніколи не       займається спортом?
13. Чи студенти, які регулярно займаються спортом, мають вищі середні результати читання, ніж ті, хто ніколи не займається спортом?

In [16]:
regularly_sports = data[data['PracticeSport'] == 'regularly']
never_sports = data[data['PracticeSport'] == 'never']

average_scores_regularly_sports = regularly_sports[['MathScore', 'ReadingScore']].mean()
average_scores_never_sports = never_sports[['MathScore', 'ReadingScore']].mean()

average_scores_regularly_sports, average_scores_never_sports

(MathScore       67.839155
 ReadingScore    69.943019
 dtype: float64,
 MathScore       64.171079
 ReadingScore    68.337662
 dtype: float64)

Проаналізуйте розподіл транспортних засобів до школи серед учнів.

14. Скільки учнів користується шкільним автобусом, щоб дістатися до школи?
15. Скільки учнів користується приватним транспортом, щоб дістатися до школи?

In [17]:
transportation_counts = data['TransportMeans'].value_counts()

transportation_counts

TransportMeans
school_bus    16145
private       11362
Name: count, dtype: int64

Задача класифікації
Передбачте, чи завершив студент курс підготовки до іспиту, на основі його демографічних та академічних даних.

Вхідні дані:
Gender, EthnicGroup, ParentEduc, LunchType, ParentMaritalStatus, PracticeSport, IsFirstChild, NrSiblings,       TransportMeans, WklyStudyHours, MathScore, ReadingScore, WritingScore

Результат: TestPrep

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


data.fillna(method='ffill', inplace=True)

X = data[['Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild', 'NrSiblings', 'TransportMeans', 'WklyStudyHours', 'MathScore', 'ReadingScore', 'WritingScore']]
y = data['TestPrep'].apply(lambda x: 1 if x == 'completed' else 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numeric_features = X.select_dtypes(include=['int', 'float']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(max_iter=1000))])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.87      0.82      6006
           1       0.67      0.50      0.57      3187

    accuracy                           0.74      9193
   macro avg       0.72      0.68      0.69      9193
weighted avg       0.73      0.74      0.73      9193

